In [23]:
import json
import math
import scipy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statistics 
import seaborn as sns
from numpy.random import randn
from statistics import mode
%matplotlib inline

In [24]:
pupils = json.load(open('pupils.json', 'r'))    # result will be a python dictionary

#jeder entry ist ein schüler: zB data[0]
len(pupils)
#2715

2996

# Klassen wählen, die das KI Paket gemacht haben

In [25]:
# taskseriesIDs vom KI Paket:
KI_taskseries_ids = ["AI-1-v2", "AI-2-v2", "AI-3-v2", "AI-4-v2", "ML-1", "ML-2", "ML-3", "Classification-1", "Clustering-1", "Classification-2", "Clustering-2", "LimitsML", "EvalML", "Impact-1", "Impact-2", "Impact-3", "Impact-4", "AIClosing"]

classIDsKI = []
pupilsKI = []

for pupilnumber in range(len(pupils)): #für jeden schüler
    for tasknumber in range(len(pupils[pupilnumber]["solvedTaskSeries"])): #für jede seiner tasks
        if("taskSeriesId" in pupils[pupilnumber]["solvedTaskSeries"][tasknumber].keys()):
            if(pupils[pupilnumber]["solvedTaskSeries"][tasknumber]["taskSeriesId"] in KI_taskseries_ids):
                for sectionnumber in range(len(pupils[pupilnumber]["solvedTaskSeries"][tasknumber]["currentSections"])): #für diese tasks, alle sections
                    for subsectionnumber in range(len(pupils[pupilnumber]["solvedTaskSeries"][tasknumber]["currentSections"][sectionnumber]["currentSubSections"])):
                        if("exercise" in pupils[pupilnumber]["solvedTaskSeries"][tasknumber]["currentSections"][sectionnumber]["currentSubSections"][subsectionnumber].keys()):
                            if pupils[pupilnumber]["classId"] not in classIDsKI:
                                classIDsKI.append(pupils[pupilnumber]["classId"])
                            if pupils[pupilnumber] not in pupilsKI:
                                pupilsKI.append(pupils[pupilnumber])

KI_classes = {classID: {"count": 0} for classID in classIDsKI}

for pupil in pupilsKI:
    KI_classes[pupil["classId"]]["count"] += 1

print(KI_classes)
print("")

# Jetzt nur die Klassen mit genügend SuS auswählen
final_pupils_KI = []
min_class_size = 10

for pupil in pupilsKI:
    if(KI_classes[pupil["classId"]]["count"] >= min_class_size):
        final_pupils_KI.append(pupil)

print("Zur Zeit gibt es " + str(len(final_pupils_KI)) + " sus, die das KI Paket bearbeitet haben")

{'uxfuiLT4xZA6yvE6L': {'count': 2}, 'qFEyvypMn4sAwLwtc': {'count': 26}, 'y7j9eKDPsqNkAycuy': {'count': 1}, 'NXCw6MMERYbnzdTjc': {'count': 1}, '6Z2MmkKQKebb7nz8P': {'count': 3}, '4fyXxCKaLbCBaPbYE': {'count': 1}, 'xYWgeSPkADwSmhdmm': {'count': 2}, 'GkQSJvPTRWsLnpyR9': {'count': 14}, '7cWBWtLEcLSYDygDc': {'count': 1}}

Zur Zeit gibt es 40 sus, die das KI Paket bearbeitet haben


## DF mit classId und pupilId konstruieren

In [26]:
class_list = [{"pupilId": pupil["_id"], "classId": pupil["classId"]} for pupil in final_pupils_KI]
class_df = pd.DataFrame(class_list)

class_df.head()

,pupilId,classId
0,bniRycMi5bwrXjvGy,qFEyvypMn4sAwLwtc
1,sqBXKssFvS8pFYAMv,qFEyvypMn4sAwLwtc
2,Kd6FuRsrq6QvPrkKp,qFEyvypMn4sAwLwtc
3,6bBwnX2mwAp2NAwNR,qFEyvypMn4sAwLwtc
4,ZTvPk2i77LDC8pFmu,qFEyvypMn4sAwLwtc


# Chat df

In [55]:
date = []
sentBy = []
messages = []
request = []

row_nr = 0
for pupil in final_pupils_KI:
    for message in pupil["chat"]["messages"]:
        date.append(message["date"])
        sentBy.append(message["sentBy"])
        messages.append(message["message"])
        if "request" in message.keys():
            request.append(message["request"])
        else:
            request.append(False)

#zipped_list = list(zip((date, sentBy, messages, request)))
#chat_df = pd.DataFrame(zipped_list, columns =["date", "sentBy", "message", "request"])

chat_df = pd.DataFrame(date)
chat_df["sentBy"] = sentBy
chat_df["message"] = messages
chat_df["request"] = request

chat_df.head()
#print(zipped_list)

auto                 266
sWm4d454H7CgZk7xP     26
eFfWFEEHYrGie6TWY     15
vPkj9gw4uYahqDz2b      2
FZTuuyvpvJBGhnCgk      1
wxkuvgoD5tbZyvyFy      1
GHpnNu3Yss7Qz8Tm4      1
Name: sentBy, dtype: int64

# Revisions DF

In [38]:
for pupil in final_pupils_KI:
    print(pupil["revisions"])

[{'_id': 'ML-3_0_1', 'taskSeriesId': 'ML-3', 'sectionIdx': 0, 'subSectionIdx': 1, 'begin': '2021-03-02T08:07:20+00:00', 'accepted': False}, {'_id': 'EvalML_0_2', 'taskSeriesId': 'EvalML', 'sectionIdx': 0, 'subSectionIdx': 2, 'begin': '2021-03-02T08:22:03+00:00', 'accepted': False}]
[{'_id': 'ML-3_0_1', 'taskSeriesId': 'ML-3', 'sectionIdx': 0, 'subSectionIdx': 1, 'begin': '2021-03-02T09:31:12+00:00', 'accepted': False}]
[]
[{'_id': 'ML-3_0_0', 'taskSeriesId': 'ML-3', 'sectionIdx': 0, 'subSectionIdx': 0, 'begin': '2021-03-02T08:46:01+00:00', 'accepted': False}, {'_id': 'ML-3_0_1', 'taskSeriesId': 'ML-3', 'sectionIdx': 0, 'subSectionIdx': 1, 'begin': '2021-03-02T08:46:39+00:00', 'accepted': False}, {'_id': 'Classification-2_0_4', 'taskSeriesId': 'Classification-2', 'sectionIdx': 0, 'subSectionIdx': 4, 'begin': '2021-03-02T08:59:37+00:00', 'accepted': False}]
[{'_id': 'AI-4-v2_0_1', 'taskSeriesId': 'AI-4-v2', 'sectionIdx': 0, 'subSectionIdx': 1, 'begin': '2021-03-02T09:34:47+00:00', 'accep

# Alle von SuS bearbeitete Aufgaben im KI Paket in einen DF

In [27]:
tasktypes = ["blockly", "multipleChoice", "freetext", "survey", "inputField", "surveySlider", "noTask"]
exercise_keys = ['state', 'submitted', 'subType', 'evaluation', 'falseAttempts', 'hintsTaken', 'rating', 'teacherFeedback', "history"]

# TODO: nehme hier alle Aufgaben von SuS, die das KI Paket bearbeitet haben, nicht nur diese Inhalte --> einschränken?

columns = ["pupilId", "taskId", "taskSeriesId", "duration"] + exercise_keys
pupil_df = pd.DataFrame(columns=columns)

row_nr = 0
for pupil in final_pupils_KI:
    for task in pupil["solvedTaskSeries"]:
        sec_nr = 1
        for section in task["currentSections"]:
            subsec_nr = 1
            for subsection in section["currentSubSections"]:
                # eine neue Zeile hinzufügen für jede bearbeitete Aufgabe und id columns füllen
                #df = df.append({"pupilId": pupil["_id"], "taskId": str(task["taskSeriesId"])+"-"+str(sec_nr)+"-"+str(subsec_nr),'taskSeriesId': task["taskSeriesId"], "sec_nr": sec_nr, "subsec_nr": subsec_nr}, ignore_index=True)
                pupil_df = pupil_df.append({"pupilId": pupil["_id"], "taskId": str(task["taskSeriesId"])+"-"+str(sec_nr)+"-"+str(subsec_nr),'taskSeriesId': task["taskSeriesId"], "duration": subsection["duration"]}, ignore_index=True)
                subsec_nr += 1
                if("exercise" in subsection.keys()):
                    # hier wird der eigentliche state der aufgabe in die reihe eingetragen
                    for key in subsection["exercise"].keys():
                        pupil_df.at[row_nr, key] = subsection["exercise"][key]
                row_nr += 1
            sec_nr +=1
    
# aufgaben loswerden, die nicht submitted wurden!
pupil_df.dropna(subset=['submitted'], inplace=True)

#die antworten der einen falschen aufgabe berichtigen
for index in pupil_df.index:
    if(pupil_df.loc[index, "taskId"] == "Classification-2-1-5"):
        if ("2" in str(pupil_df.loc[index, "falseAttempts"])): #die SuS, die es eigentlich richtig hatten
            pupil_df.loc[index, "falseAttempts"] = np.nan
            pupil_df.loc[index, "evaluation"] = "correct"
        elif(pupil_df.loc[index, "pupilId"]=="gooproPtWeTH6dtcB"): #der eine schüler, der es falsch hatte
            pupil_df.loc[index, "falseAttempts"] = [str(pupil_df.loc[index, "state"])]
            pupil_df.loc[index, "evaluation"] = "false"

pupil_df.head()


,pupilId,taskId,taskSeriesId,duration,state,submitted,subType,evaluation,falseAttempts,hintsTaken,rating,teacherFeedback,history
0,bniRycMi5bwrXjvGy,AI-1-v2-1-1,AI-1-v2,40,92,True,surveySlider,NaN,NaN,NaN,NaN,NaN,[]
1,bniRycMi5bwrXjvGy,AI-1-v2-1-2,AI-1-v2,119,"<p>Künstliche Intelligenz ist ein Algorithmus,...",True,freetext,correct,NaN,NaN,"{'content': 3, 'format': 5, 'creativity': 4}","Inhaltlich gut, etwas genauer antworten. Muste...",[]
2,bniRycMi5bwrXjvGy,AI-1-v2-2-1,AI-1-v2,268,"[0, 0, 1, 0, 0]",True,survey,NaN,NaN,NaN,NaN,NaN,[]
4,bniRycMi5bwrXjvGy,AI-2-v2-1-1,AI-2-v2,94,"[True, True, False, True]",True,multipleChoice,correct,NaN,NaN,NaN,NaN,[]
6,bniRycMi5bwrXjvGy,AI-3-v2-1-1,AI-3-v2,43,"[False, False, False]",True,multipleChoice,correct,NaN,NaN,NaN,NaN,[]


## DF mit taskSeries bauen

In [28]:
#taskSeries Laden
taskSeries = json.load(open('taskSeries.json', 'r'))    # result will be a python dictionary

In [16]:
#KI_taskseries_ids = ["AI-1-v2", "AI-2-v2", "AI-3-v2", "AI-4-v2", "ML-1", "ML-2", "ML-3", "Classification-1", "Clustering-1", "Classification-2", "Clustering-2", "LimitsML", "EvalML", "Impact-1", "Impact-2", "Impact-3", "Impact-4", "AIClosing"]

interaction_subtypes = ['multipleChoice', 'blockly', 'surveySlider', 'survey', 'inputField', 'freetext']
columns = ["taskId", "question", "answers", "minLabel", "maxLabel", "hints", "sampleSolution"]
task_df = pd.DataFrame(columns=columns)

row_nr = 0
for taskSerie in taskSeries:
    sec_nr = 1
    if taskSerie["_id"] in KI_taskseries_ids:
        for section in taskSerie["sections"]:
            subsec_nr = 1
            for subsection in section["subSections"]:
                for elem in subsection["elements"]:
                    if(elem["subType"] in interaction_subtypes):
                        task_df = task_df.append({"taskId": str(taskSerie["_id"])+"-"+str(sec_nr)+"-"+str(subsec_nr), "question": elem["question"]["de"]}, ignore_index=True)
                        #task_df = task_df.append({"taskId": str(taskSerie["_id"])+"-"+str(sec_nr)+"-"+str(subsec_nr),'taskSeriesId': taskSerie["_id"], "sec_nr": sec_nr, "subsec_nr": subsec_nr, "subType": elem["subType"], "question": elem["question"]["de"]}, ignore_index=True)
                        if(elem["subType"] == "surveySlider"):
                            task_df.at[row_nr, "minLabel"] = elem["specificData"]["minLabel"]["de"]
                            task_df.at[row_nr, "maxLabel"] = elem["specificData"]["maxLabel"]["de"]
                        elif(elem["subType"] == "freetext"):
                            task_df.at[row_nr, "sampleSolution"] = elem["sampleSolution"]["de"]
                        elif(elem["subType"] == "survey"):
                            task_df.at[row_nr, "answers"] = elem["specificData"]["answers"]
                        elif(elem["subType"] == "multipleChoice"):
                            task_df.at[row_nr, "answers"] = elem["specificData"]["answers"]
                            task_df.at[row_nr, "sampleSolution"] = elem["sampleSolution"]["universal"]
                            if(len(elem["hints"]) > 0):
                                task_df.at[row_nr, "hints"] = elem["hints"][0]["de"]
                        elif(elem["subType"] == "inputField"):
                            task_df.at[row_nr, "sampleSolution"] = elem["sampleSolution"]["universal"]
                            if(len(elem["hints"]) > 0):
                                task_df.at[row_nr, "hints"] = elem["hints"][0]["de"]
                        row_nr += 1
                subsec_nr +=1
            sec_nr +=1
            
task_df.head()

,taskId,question,answers,minLabel,maxLabel,hints,sampleSolution
0,AI-1-v2-1-1,"Wie gut weißt du schon, was KI ist?",NaN,Gar nicht,Sehr gut,NaN,NaN
1,AI-1-v2-1-2,Was ist deiner Meinung nach Künstliche Intelli...,NaN,NaN,NaN,NaN,Die Schülerinnen und Schüler (SuS) sollen besc...
2,AI-1-v2-2-1,Wie viele deiner Zeichnungen hat die KI erkann...,"[{'de': '0', 'en-US': '0'}, {'de': '1-2', 'en-...",NaN,NaN,NaN,NaN
3,AI-2-v2-1-1,"Kannst du erkennen, wo KI überall drin steckt?...",[{'de': 'Der Newsfeed eines Social Network nut...,NaN,NaN,"Denk an die Beispiele, die wir uns am Anfang a...","[True, True, False, True]"
4,AI-3-v2-1-1,Was für Sachen können KI-Anwendungen?,"[{'de': 'Die Anwendung, die den Facebook-Feed ...",NaN,NaN,"Kann überhaupt eine dieser Anwendungen etwas, ...","[False, False, False]"


## DF mergen

In [29]:
new_df = pd.merge(pupil_df, task_df, on='taskId')
new_df.drop(columns=['history'], inplace=True)

In [30]:
df = pd.merge(class_df, new_df, on="pupilId")
df.head()

,pupilId,classId,taskId,taskSeriesId,duration,state,submitted,subType,evaluation,falseAttempts,hintsTaken,rating,teacherFeedback,question,answers,minLabel,maxLabel,hints,sampleSolution
0,bniRycMi5bwrXjvGy,qFEyvypMn4sAwLwtc,AI-1-v2-1-1,AI-1-v2,40,92,True,surveySlider,NaN,NaN,NaN,NaN,NaN,"Wie gut weißt du schon, was KI ist?",NaN,Gar nicht,Sehr gut,NaN,NaN
1,bniRycMi5bwrXjvGy,qFEyvypMn4sAwLwtc,AI-1-v2-1-2,AI-1-v2,119,"<p>Künstliche Intelligenz ist ein Algorithmus,...",True,freetext,correct,NaN,NaN,"{'content': 3, 'format': 5, 'creativity': 4}","Inhaltlich gut, etwas genauer antworten. Muste...",Was ist deiner Meinung nach Künstliche Intelli...,NaN,NaN,NaN,NaN,Die Schülerinnen und Schüler (SuS) sollen besc...
2,bniRycMi5bwrXjvGy,qFEyvypMn4sAwLwtc,AI-1-v2-2-1,AI-1-v2,268,"[0, 0, 1, 0, 0]",True,survey,NaN,NaN,NaN,NaN,NaN,Wie viele deiner Zeichnungen hat die KI erkann...,"[{'de': '0', 'en-US': '0'}, {'de': '1-2', 'en-...",NaN,NaN,NaN,NaN
3,bniRycMi5bwrXjvGy,qFEyvypMn4sAwLwtc,AI-2-v2-1-1,AI-2-v2,94,"[True, True, False, True]",True,multipleChoice,correct,NaN,NaN,NaN,NaN,"Kannst du erkennen, wo KI überall drin steckt?...",[{'de': 'Der Newsfeed eines Social Network nut...,NaN,NaN,"Denk an die Beispiele, die wir uns am Anfang a...","[True, True, False, True]"
4,bniRycMi5bwrXjvGy,qFEyvypMn4sAwLwtc,AI-3-v2-1-1,AI-3-v2,43,"[False, False, False]",True,multipleChoice,correct,NaN,NaN,NaN,NaN,Was für Sachen können KI-Anwendungen?,"[{'de': 'Die Anwendung, die den Facebook-Feed ...",NaN,NaN,"Kann überhaupt eine dieser Anwendungen etwas, ...","[False, False, False]"


## Pupil performance df

In [31]:
nr_tasks_per_pupil = []
total_duration_per_pupil = []

for pupilId in df["pupilId"].unique():
    nr_tasks_per_pupil.append(len(df[df["pupilId"]==pupilId]))
    total_duration_per_pupil.append(sum(df[df["pupilId"]==pupilId]["duration"])/60)

pupil_performance = {"pupilId": df["pupilId"].unique(), "nr_tasks_per_pupil": nr_tasks_per_pupil, "total_duration in minutes": total_duration_per_pupil}
pupil_performance_df = pd.DataFrame(pupil_performance)
pupil_performance_df["average duration per task in minutes"] = pupil_performance_df["total_duration in minutes"]/pupil_performance_df["nr_tasks_per_pupil"]

pupil_performance_df.head()

,pupilId,nr_tasks_per_pupil,total_duration in minutes,average duration per task in minutes
0,bniRycMi5bwrXjvGy,67,71.083333,1.060945
1,sqBXKssFvS8pFYAMv,67,66.966667,0.999502
2,Kd6FuRsrq6QvPrkKp,6,37.383333,6.230556
3,6bBwnX2mwAp2NAwNR,36,116.100000,3.225000
4,ZTvPk2i77LDC8pFmu,67,60.466667,0.902488


## Saving the DFs as csvs

In [21]:
pupil_performance_df.to_csv('pupil_performance.csv')
df.to_csv('df.csv')

In [33]:
storedf = pd.HDFStore("store.h5")
storedf["df"] = df

/Users/jean-philipp/Documents/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3417: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block0_values] [items->Index(['pupilId', 'classId', 'taskId', 'taskSeriesId', 'duration', 'state',
       'submitted', 'subType', 'evaluation', 'falseAttempts', 'hintsTaken',
       'rating', 'teacherFeedback', 'question', 'answers', 'minLabel',
       'maxLabel', 'hints', 'sampleSolution'],
      dtype='object')]

  exec(code_obj, self.user_global_ns, self.user_ns)


In [65]:
pupil_performance_df.to_pickle('pupil_performance.pkl')
df.to_pickle('df.pkl')
chat_df.to_pickle("chat.pkl")